In [9]:
from ortools.linear_solver import pywraplp
import csv
from pathlib import Path
import os

### Carrega os dados dos arquivos
O código a seguir carrega os dados para o experimento.

In [12]:
def otimizaSprints(dados, cap_sprint, cap_necessaria_backlog):
    # garante que a quantidade de requisitos, pesos e valores são iguais
    assert len(dados['requisitos']) == len(dados['valores']) == len(dados['pesos'])
    dados['num_items'] = len(dados['requisitos'])

    num_sprints = round((cap_necessaria_backlog / cap_sprint) + 0.5)
    dados['bin_capacities'] = []
    for i in range(num_sprints):
        dados['bin_capacities'].append(cap_sprint)

    dados['num_bins'] = num_sprints
    dados['all_items'] = range(dados['num_items'])
    dados['all_bins'] = range(dados['num_bins'])

    solver = pywraplp.Solver.CreateSolver('SCIP')
    if solver is None:
        print('SCIP solver unavailable.')
        return

    # x[i, b] = 1 if item i is packed in bin b.
    x = {}
    for i in dados['all_items']:
        for b in dados['all_bins']:
            x[i, b] = solver.BoolVar(f'x_{i}_{b}')

    # Each item is assigned to at most one bin.
    for i in dados['all_items']:
        solver.Add(sum(x[i, b] for b in dados['all_bins']) <= 1)

    # The amount packed in each bin cannot exceed its capacity.
    for b in dados['all_bins']:
        solver.Add(
            sum(x[i, b] * dados['pesos'][i]
                for i in dados['all_items']) <= dados['bin_capacities'][b])

    # Maximize total value of packed items.
    objective = solver.Objective()
    for i in dados['all_items']:
        for b in dados['all_bins']:
            objective.SetCoefficient(x[i, b], dados['valores'][i])
    objective.SetMaximization()

    status = solver.Solve()

    if status == pywraplp.Solver.OPTIMAL:
        print(f'Valor total embalado: {objective.Value()}')
        total_weight = 0
        for b in dados['all_bins']:
            print(f'Sprint {b}')
            bin_weight = 0
            bin_value = 0
            for i in dados['all_items']:
                if x[i, b].solution_value() > 0:
                    print(
                        f"Requisito {i} peso: {dados['pesos'][i]} valor: {dados['valores'][i]}"
                    )
                    bin_weight += dados['pesos'][i]
                    bin_value += dados['valores'][i]
            print(f'Peso da sprint: {bin_weight}')
            print(f'Valor da sprint: {bin_value}\n')
            total_weight += bin_weight
        print(f'Peso total embalado: {total_weight}')
    else:
        print('O problema não tem uma solução ótima.')

In [13]:
# recupera o caminho completo do diretório de dados
dir_dados = str(Path(os.getcwd()).parents[0]) + '\data'

# percorre cada arquivo presente no diretório de dados
for arquivo in Path(dir_dados).iterdir():
    # lista que armazenará os dados de cada arquivo
    dados = {}
    requisitos = []
    valores = []
    pesos = []

    if arquivo.is_file() and arquivo.suffix == '.csv':
        # faz a leitura do CSV
        with open(arquivo, newline='') as csvfile:
            leitor = csv.DictReader(csvfile)
            for linha in leitor:
                requisitos.append(linha['requisito'])
                valores.append(float(linha['valor']))
                pesos.append(float(linha['peso']))

        dados['requisitos'] = requisitos
        dados['valores'] = valores
        dados['pesos'] = pesos

    cap_necessaria_backlog = 0
    for peso in dados['pesos']:
        cap_necessaria_backlog += peso

    # chama função que realiza a otimização
    otimizaSprints(dados, 180, cap_necessaria_backlog)

Valor total embalado: 109.81621613603163
Sprint 0
Requisito 0 peso: 32.28053209435794 valor: 11.210266392610656
Requisito 1 peso: 24.208696222551453 valor: 16.482490791690317
Requisito 2 peso: 22.423910681461532 valor: 17.920377822913572
Requisito 3 peso: 16.83838089802525 valor: 10.76543275611364
Requisito 4 peso: 12.615513941983249 valor: 14.148752931385605
Requisito 5 peso: 18.59225356603736 valor: 9.350409933902045
Requisito 6 peso: 13.72113120953664 valor: 3.459123869919255
Requisito 7 peso: 12.456280585442318 valor: 5.565329429625209
Requisito 8 peso: 8.775170817278664 valor: 3.2738799555325566
Peso da sprint: 161.9118700166744
Valor da sprint: 92.17606388369285

Sprint 1
Requisito 9 peso: 34.67512165570563 valor: 17.640152252338773
Peso da sprint: 34.67512165570563
Valor da sprint: 17.640152252338773

Peso total embalado: 196.58699167238004
Valor total embalado: 980.9445255145686
Sprint 0
Requisito 0 peso: 20.031966142521995 valor: 8.297027778644395
Requisito 1 peso: 24.00849419